In [1]:

import pyomo.environ as pe
import math

# Data
factories = ["IJmuiden", "Segal", "South Wales"]
rebars = {"A": {"length": 2.4}, "B": {"length": 3.6}, "C": {"length": 4.2}}
longbars = {1: {"length": 9}, 2: {"length": 12}}
diameter = 0.057  # in meters
density = 7.85  # tons/m^3
production_capacity = {"IJmuiden": 12, "Segal": 10, "South Wales": 28}
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]
periods = [1, 2, 3, 4]

demand = {
    "A": {
        "Bochum": [2, 6, 5, 3],
        "Boenen": [4, 8, 5, 10],
        "Dortmund": [2, 7, 6, 5],
        "Gelsenkirchen": [5, 5, 5, 5],
        "Hagen": [19, 23, 25, 16],
        "Iserlohn": [13, 19, 17, 14],
        "Neuss": [20, 16, 14, 26],
        "Schwerte": [4, 5, 3, 4]
    },
    "B": {
        "Bochum": [4, 5, 7, 8],
        "Boenen": [5, 8, 12, 13],
        "Dortmund": [4, 5, 8, 10],
        "Gelsenkirchen": [9, 10, 6, 6],
        "Hagen": [15, 33, 31, 33],
        "Iserlohn": [22, 26, 20, 27],
        "Neuss": [12, 23, 30, 30],
        "Schwerte": [2, 8, 2, 6]
    },
    "C": {
        "Bochum": [6, 7, 7, 7],
        "Boenen": [6, 10, 15, 12],
        "Dortmund": [7, 6, 4, 12],
        "Gelsenkirchen": [10, 9, 9, 10],
        "Hagen": [12, 35, 33, 38],
        "Iserlohn": [14, 25, 23, 24],
        "Neuss": [22, 32, 31, 31],
        "Schwerte": [5, 6, 7, 2]
    }
}

fixed_cost = {"IJmuiden": 130, "Segal": 150, "South Wales": 100}
v = 0.5  # variable cost €/km/tonne
distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Precompute parameters
w_r = {r: rebars[r]["length"] for r in rebars}
w_l = {l: longbars[l]["length"] for l in longbars}
area = math.pi * (diameter / 2) ** 2


# possible number of long bars per long bar type per factory 
N = {
    1: {"IJmuiden": 67, "Segal": 56, "South Wales": 156},
    2: {"IJmuiden": 50, "Segal": 42, "South Wales": 117}
}

possible_longbars = {} 

for l in longbars:
    possible_longbars[l] = {} 
    for f in factories:
        possible_longbars[l][f] = list(range(1, N.get(l, {}).get(f, 0) + 1))

# Defining big M
# List to store total demand per customer
total_demand_per_customer = []

# Loop over each customer
for a in customers:
    total_demand = sum(demand[r][a][t-1] for r in rebars for t in periods)  # Sum over all rebars and periods
    total_demand_per_customer.append(total_demand)

# Get the maximum demand across all customers
M = max(total_demand_per_customer)
  

In [5]:
# Question 1
# Create the model
model = pe.ConcreteModel()

# Define sets
model.F = pe.Set(initialize=factories)
model.R = pe.Set(initialize=rebars.keys())
model.L = pe.Set(initialize=longbars.keys())
model.A = pe.Set(initialize=customers)
model.T = pe.Set(initialize=periods)


valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model.x = pe.Var(model.R, model.L_N_F, model.T, domain=pe.NonNegativeIntegers)
model.y = pe.Var(model.L_N_F, model.T, domain=pe.Binary)
model.z = pe.Var(model.F, model.A, model.T, domain=pe.Binary)

# Constraints using ConstraintList and for loops
# 1. Cutting constraints
model.cutting_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        for l in model.L:
            for n in possible_longbars[l][f]:
                expression = sum(model.x[r, l, n, f, t] * w_r[r] for r in model.R) <= model.y[l, n, f, t] * w_l[l]
                model.cutting_cnstr.add(expression)

# 2. Production capacity
model.capacity_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        expression = sum(model.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model.L) * density * area <= production_capacity[f]
        model.capacity_cnstr.add(expression)

# 3. Demand satisfaction
model.demand_cnstr = pe.ConstraintList()
for f in model.F:
    for t in model.T:
        for r in model.R:
            expression = sum(model.x[r, l, n, f, t] for n in possible_longbars[l][f] for l in model.L) >= sum(model.z[f, a, t] * demand[r][a][t-1] for a in model.A)
            model.demand_cnstr.add(expression)

# 4. Single sourcing
model.single_sourcing_cnstr = pe.ConstraintList()
for a in model.A:
    for t in model.T:
        expression = sum(model.z[f, a, t] for f in model.F) == 1
        model.single_sourcing_cnstr.add(expression)

# Objective function
objExpr = sum(model.z[f, a, t] * (fixed_cost[f] +  v * distance[a][f] * sum(demand[r][a][t-1] * w_r[r] for r in model.R) * density * area) for f in model.F for a in model.A for t in model.T)  
model.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result = solver.solve(model, tee=True, options={'TimeLimit':36})

# Display results
print(f"\nSolver Status: {result.solver.status}")
print(f"Termination Condition: {result.solver.termination_condition}")
if result.solver.status == pe.SolverStatus.ok and result.solver.termination_condition == pe.TerminationCondition.optimal:
    print(f"Objective value (Total Cost): {pe.value(model.obj):.2f} €")
    print("\nFactory assignments (z[f,a,t] = 1):")
    for f in model.F:
        for a in model.A:
            for t in model.T:
                if pe.value(model.z[f, a, t]) > 0.5:
                    print(f"z[{f}, {a}, {t}] = 1")
    print("\nRebars cut (x[r,l,n,f,t] > 0):")
    for r in model.R:
        for l in model.L:
            for f in model.F:
                for n in possible_longbars[l][f]:
                    for t in model.T:
                        if pe.value(model.x[r, l, n, f, t]) > 0:
                            print(f"x[{r}, {l}, {n}, {f}, {t}] = {pe.value(model.x[r, l, n, f, t]):.0f}")
    print("\nLong bars used (y[l,n,f,t] > 0):")
    for l in model.L:
        for f in model.F:
            for n in possible_longbars[l][f]:
                for t in model.T:
                    if pe.value(model.y[l, n, f, t]) > 0:
                        print(f"y[{l}, {n}, {f}, {t}] = {pe.value(model.y[l, n, f, t]):.0f}")
else:
    print("No optimal solution found.")

Read LP format model from file /var/folders/3y/y2gscvn90gjfj79j_xc13zyr0000gp/T/tmpohklhffd.pyomo.lp
Reading time = 0.01 seconds
x1: 2032 rows, 7904 columns, 14880 nonzeros
Set parameter TimeLimit to value 36
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  36

Optimize a model with 2032 rows, 7904 columns and 14880 nonzeros
Model fingerprint: 0x1d4d1c58
Variable types: 0 continuous, 7904 integer (2048 binary)
Coefficient statistics:
  Matrix range     [2e-01, 4e+01]
  Objective range  [2e+02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 718 rows and 2816 columns
Presolve time: 0.36s
Presolved: 1314 rows, 5088 columns, 10320 nonzeros
Variable types: 0 continuous, 5088 integer (1326 binary)

Root relaxation: objective 1.612448e+04, 2037 iterations, 0.01 seconds (0.02 work uni

In [2]:
# New data for 2: Inventory capacities (from Table 7)
inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}

In [ ]:
# New data for 2: Inventory capacities (from Table 7)
inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}

model2 = pe.ConcreteModel()

# Initialize sets for model 2
model2.F = pe.Set(initialize=factories)
model2.R = pe.Set(initialize=rebars.keys())
model2.L = pe.Set(initialize=longbars.keys())
model2.A = pe.Set(initialize=customers)
model2.T = pe.Set(initialize=periods)

valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model2.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model2.x = pe.Var(model2.R, model2.L_N_F,  model2.T, domain=pe.NonNegativeIntegers)
model2.y = pe.Var(model2.L_N_F, model2.T, domain=pe.Binary)
model2.z = pe.Var(model2.F, model2.A, model2.T, domain=pe.Binary)

# Add new variables to model2
model2.i = pe.Var(model2.A, model2.R, model2.T, domain=pe.NonNegativeIntegers)
model2.q = pe.Var(model2.R, model2.F, model2.A, model2.T, domain=pe.NonNegativeIntegers)


# Constraints 
# 1. Cutting constraints - same as Q1
model2.cutting_cnstr = pe.ConstraintList()
for f in model2.F:
    for t in model2.T:
        for l in model2.L:
            for n in possible_longbars[l][f]:
                expression = sum(model2.x[r, l, n, f, t] * w_r[r] for r in model2.R) <= model2.y[l, n, f, t] * w_l[l]
                model2.cutting_cnstr.add(expression)

# 2. Production capacity - same as Q1
model2.capacity_cnstr = pe.ConstraintList()
for f in model2.F:
    for t in model2.T:
        expression = sum(model2.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model2.L) * density * area <= production_capacity[f]
        model2.capacity_cnstr.add(expression)

# Factory output
model2.factory_output = pe.ConstraintList()
for f in model2.F:
    for r in model2.R:
        for t in model2.T:
            expression = sum(model2.q[(r, f, a, t)] for a in model2.A) == sum(model2.x[(r, l, n, f, t)] for n in possible_longbars[l][f] for l in model2.L)
            model2.factory_output.add(expression)

# Link z to q
model2.enforce_factory_usage = pe.ConstraintList()
for f in model2.F:
    for a in model2.A:
        for t in model2.T:
            expression = sum(model2.q[(r, f, a, t)] for r in model2.R) <= M * model2.z[f, a, t]
            model2.enforce_factory_usage.add(expression)

# Demand 
model2.demand = pe.ConstraintList()
for a in model2.A:
    for r in model2.R:
        for t in model2.T:
            shipped = sum(model2.q[(r, f, a, t)] for f in model2.F) 
            if t == 1:
                expression = shipped - model2.i[(a,r,t)] == demand[r][a][t-1] 
                model2.demand.add(expression)
            else:
                expression = shipped - model2.i[(a,r,t)] + model2.i[(a,r,t-1)] == demand[r][a][t-1] 
                model2.demand.add(expression)

# Single sourcing
model2.single_sourcing_cnstr = pe.ConstraintList()
for a in model2.A:
    for t in model2.T:
        expression = sum(model2.z[(f,a,t)] for f in model2.F) <= 1
        model2.single_sourcing_cnstr.add(expression)

# Inventory
model2.inventory = pe.ConstraintList()
for a in model2.A:
    for t in model2.T:
        expression = sum(model2.i[(a,r,t)]* w_r[r] for r in model2.R) * density * area <= inventory_capacity[a]
        model2.inventory.add(expression)


# Objective function
objExpr = sum(model2.z[f, a, t] * (fixed_cost[f] +  v * distance[a][f] * sum(model2.q[(r, f, a, t)] * w_r[r] for r in model2.R) * density * area) for f in model2.F for a in model2.A for t in model2.T)  
model2.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result2 = solver.solve(model2, tee=True, options={'TimeLimit':3600})

# Display results
print(f"\nSolver Status: {result2.solver.status}")
print(f"Termination Condition: {result2.solver.termination_condition}")
if result2.solver.status == pe.SolverStatus.ok and result2.solver.termination_condition == pe.TerminationCondition.optimal:
    print(f"Objective value (Total Cost): {pe.value(model2.obj):.2f} €")
    print("\nFactory assignments (z[f,a,t] = 1):")
    for f in model2.F:
        for a in model2.A:
            for t in model2.T:
                if pe.value(model2.z[f, a, t]) > 0.5:
                    print(f"z[{f}, {a}, {t}] = 1")
    print("\nQuantities shipped (q[r,f,a,t] > 0):")
    for r in model2.R:
        for f in model2.F:
            for a in model2.A:
                for t in model2.T:
                    if pe.value(model2.q[r,f,a,t]) > 0:
                        print(f"q[{r}, {f}, {a}, {t}] = {pe.value(model2.q[r,f,a,t]):.0f}")
    print("\nRebars cut (x[r,l,n,f,t] > 0):")
    for r in model2.R:
        for l in model2.L:
            for f in model2.F:
                for n in possible_longbars[l][f]:
                    for t in model2.T:
                        if pe.value(model2.x[r, l, n, f, t]) > 0:
                            print(f"x[{r}, {l}, {n}, {f}, {t}] = {pe.value(model2.x[r, l, n, f, t]):.0f}")
    print("\nLong bars used (y[l,n,f,t] > 0):")
    for l in model2.L:
        for f in model2.F:
            for n in possible_longbars[l][f]:
                for t in model2.T:
                    if pe.value(model2.y[l, n, f, t]) > 0:
                        print(f"y[{l}, {n}, {f}, {t}] = {pe.value(model2.y[l, n, f, t]):.0f}")
    
else:
    print("No optimal solution found.")


Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpscjw64mk.pyomo.lp
Reading time = 0.04 seconds
x1: 2256 rows, 8288 columns, 15816 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 2256 rows, 8288 columns and 15816 nonzeros
Model fingerprint: 0x4f785dd6
Model has 288 quadratic objective terms
Variable types: 0 continuous, 8288 integer (2048 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+02]
  Objective range  [1e+02, 2e+02]
  QObjective range [7e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 280 rows and 1120 columns
Presolve time: 0.08s
Presolved: 2264 rows, 7456 columns, 15560 nonzeros
Variable types: 0 continuous, 74

In [3]:
#New data for 3: 
distanceq3 = {
    "IJmuiden": {"IJmuiden": 0, "Segal": 284, "South Wales": 826, "Bochum": 250, "Boenen": 282, "Dortmund": 266, "Gelsenkirchen": 234, "Hagen": 289, "Iserlohn": 299, "Neuss": 259, "Schwerte": 279},
    "Segal": {"IJmuiden": 284, "Segal": 0, "South Wales": 750, "Bochum": 203, "Boenen": 242, "Dortmund": 222, "Gelsenkirchen": 198, "Hagen": 206, "Iserlohn": 226, "Neuss": 140, "Schwerte": 216},
    "South Wales": {"IJmuiden": 826, "Segal": 750, "South Wales": 0, "Bochum": 866, "Boenen": 914, "Dortmund": 885, "Gelsenkirchen": 859, "Hagen": 903, "Iserlohn": 913, "Neuss": 843, "Schwerte": 901},
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866, "Bochum": 0, "Boenen": 55, "Dortmund": 21, "Gelsenkirchen": 19, "Hagen": 41, "Iserlohn": 51, "Neuss": 56, "Schwerte": 39},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914, "Bochum": 55, "Boenen": 0, "Dortmund": 34, "Gelsenkirchen": 56, "Hagen": 44, "Iserlohn": 54, "Neuss": 102, "Schwerte": 32},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885, "Bochum": 21, "Boenen": 34, "Dortmund": 0, "Gelsenkirchen": 34, "Hagen": 21, "Iserlohn": 36, "Neuss": 75, "Schwerte": 15},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859, "Bochum": 19, "Boenen": 56, "Dortmund": 34, "Gelsenkirchen": 0, "Hagen": 56, "Iserlohn": 66, "Neuss": 62, "Schwerte": 54},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903, "Bochum": 41, "Boenen": 44, "Dortmund": 21, "Gelsenkirchen": 56, "Hagen": 0, "Iserlohn": 20, "Neuss": 66, "Schwerte": 14},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913, "Bochum": 51, "Boenen": 54, "Dortmund": 36, "Gelsenkirchen": 66, "Hagen": 20, "Iserlohn": 0, "Neuss": 85, "Schwerte": 15},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843, "Bochum": 56, "Boenen": 102, "Dortmund": 75, "Gelsenkirchen": 62, "Hagen": 66, "Iserlohn": 85, "Neuss": 0, "Schwerte": 75},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901, "Bochum": 39, "Boenen": 32, "Dortmund": 15, "Gelsenkirchen": 54, "Hagen": 14, "Iserlohn": 15, "Neuss": 75, "Schwerte": 0}
}

#Vehicle capacity
Q = 25
# Cost per km
C = 3

model3 = pe.ConcreteModel()

# Initialize sets for model 2
model3.F = pe.Set(initialize=factories)
model3.R = pe.Set(initialize=rebars.keys())
model3.L = pe.Set(initialize=longbars.keys())
model3.A = pe.Set(initialize=customers)
model3.T = pe.Set(initialize=periods)
model3.F_A = model3.F | model3.A
model3.A_f = pe.Set(model3.F, initialize=lambda model3, f: model3.A | {f})
#model3.C = pe.Set(model3.A, model3.T, within=model3.A, initialize=lambda model, a, t: [])


valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model3.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model3.x = pe.Var(model3.R, model3.L_N_F,  model3.T, domain=pe.NonNegativeIntegers)
model3.y = pe.Var(model3.L_N_F, model3.T, domain=pe.Binary)
#model3.z = pe.Var(model3.F, model3.A, model3.T, domain=pe.Binary, initialize = 0)
model3.i = pe.Var(model3.A, model3.R, model3.T, domain=pe.NonNegativeIntegers)
model3.q = pe.Var(model3.R, model3.F, model3.A, model3.T, domain=pe.NonNegativeIntegers, initialize = 0)

# Define the valid indices for model3.v dynamically
#valid_v_indices = [(f, A_f, A_f, t) for f in model3.F for t in model3.T for A_f in model3.A_f[f]]
valid_v_indices = [
    (f, A_f_source, A_f_dest, t)
    for f in model3.F
    for t in model3.T
    for A_f_source in model3.A_f[f]
    for A_f_dest in model3.A_f[f]
]

# Add new variables to model3
model3.v = pe.Var(valid_v_indices, domain=pe.Binary, initialize = 0)
model3.u = pe.Var(model3.A, model3.F, model3.T, domain=pe.NonNegativeIntegers, initialize = 0)

# Add notation for t_fat and z_fat for simplicity
# Define t_fat as the tonnes shipped from factory f to customer a in period t
model3.t = pe.Expression(model3.F, model3.A, model3.T, 
                                  rule=lambda model3, f, a, t: 
                                  sum(model3.q[(r, f, a, t)] * w_r[r] * density * area for r in model3.R))
      
model3.z = pe.Expression(model3.F, model3.A, model3.T, 
                                  rule=lambda model3, f, a, t: 
                                  sum(model3.v[(f, i, a, t)] for i in model3.A_f[f]))
  

# Constraints 
# 1. Cutting constraints - same as Q1
model3.cutting_cnstr = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for l in model3.L:
            for n in possible_longbars[l][f]:
                expression = sum(model3.x[r, l, n, f, t] * w_r[r] for r in model3.R) <= model3.y[l, n, f, t] * w_l[l]
                model3.cutting_cnstr.add(expression)

# 2. Production capacity - same as Q1
model3.capacity_cnstr = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        expression = sum(model3.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model3.L) * density * area <= production_capacity[f]
        model3.capacity_cnstr.add(expression)

# Factory output
model3.factory_output = pe.ConstraintList()
for f in model3.F:
    for r in model3.R:
        for t in model3.T:
            expression = sum(model3.q[(r, f, a, t)] for a in model3.A) == sum(model3.x[(r, l, n, f, t)] for n in possible_longbars[l][f] for l in model3.L)
            model3.factory_output.add(expression)

# Link z to q
model3.enforce_factory_usage = pe.ConstraintList()
for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            expression = sum(model3.q[(r, f, a, t)] for r in model3.R) <= M * model3.z[f, a, t]
            model3.enforce_factory_usage.add(expression)

# Demand 
model3.demand = pe.ConstraintList()
for a in model3.A:
    for r in model3.R:
        for t in model3.T:
            shipped = sum(model3.q[(r, f, a, t)] for f in model3.F) 
            if t == 1:
                expression = shipped - model3.i[(a,r,t)] == demand[r][a][t-1] 
                model3.demand.add(expression)
            else:
                expression = shipped - model3.i[(a,r,t)] + model3.i[(a,r,t-1)] == demand[r][a][t-1] 
                model3.demand.add(expression)

# Single sourcing
model3.single_sourcing_cnstr = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.z[(f,a,t)] for f in model3.F) <= 1
        model3.single_sourcing_cnstr.add(expression)

# Inventory
model3.inventory = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.i[(a,r,t)]* w_r[r] for r in model3.R) * density * area <= inventory_capacity[a]
        model3.inventory.add(expression)

# New constraints
                              
# Flow conservation
model3.flow_conservation = pe.ConstraintList()
for f in model3.F:
    for j in model3.A_f[f]:
        for t in model3.T:
            expression = sum(model3.v[(f,i,j,t)] for i in model3.A_f[f]) == sum(model3.v[(f,j,h,t)] for h in model3.A_f[f])
            model3.flow_conservation.add(expression)

# Subtour Elimination Constraint
model3.subtour_elimination = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A: 
            for j in model3.A:
                if i != j:
                    expression = model3.u[i,f,t] + model3.t[f,j,t] <= model3.u[j,f,t] + Q * (1 - model3.v[(f,i, j, t)])
                    model3.subtour_elimination.add(expression)

# Limiting t
model3.upper_bound_t = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A:
            expression = model3.t[f,i,t] <= model3.u[i,f,t]
           # model3.upper_bound_t.add(expression)

# don't go from i to i
model3.prevent_loops = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A_f[f]:
            expression = model3.v[(f,i, i, t)] == 0
            model3.prevent_loops.add(expression)


# Limiting u
model3.upper_bound_u = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A:
            expression = model3.u[i,f,t] <= Q
            model3.upper_bound_u.add(expression)

# Each customer a is visited if z_fat=1 (link v to z)
model3.visit_each_customer = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for j in model3.A:
            expression = sum(model3.v[(f,i, j, t)] for i in model3.A_f[f]) == model3.z[f,j,t]
            model3.visit_each_customer.add(expression)

# Vehicle capacity
model3.vehicle_capacity = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        expression = sum(model3.t[f,a,t] for a in model3.A) <= Q
        model3.vehicle_capacity.add(expression)


# Objective function
objExpr = sum(model3.z[f, a, t] * C * sum(model3.v[(f,i, a, t)] * distanceq3.get(i, {}).get(a, 0) for i in model3.A_f[f])for f in model3.F for a in model3.A for t in model3.T)  
model3.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result3 = solver.solve(model3, tee=True, options={'TimeLimit':3600})

print(f"\nSolver Status: {result3.solver.status}")
print(f"Termination Condition: {result3.solver.termination_condition}")
print(f"Objective value (Total Cost): {pe.value(model3.obj):.2f} €")

#debugging
for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            for r in model3.R:
                if pe.value(model3.q[r, f, a, t]) > 0:
                    print(f"Rebar {r} is shipped from {f} to {a} in period {t} with quantity {pe.value(model3.q[r, f, a, t])}")

for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            if pe.value(model3.z[f, a, t]) > 0:
                print(f"Factory {f} is assigned to customer {a} in period {t}")


Set parameter Username
Set parameter LicenseID to value 2618603
Academic license - for non-commercial use only - expires 2026-02-05
Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpund50ksv.pyomo.lp
Reading time = 0.05 seconds
x1: 3348 rows, 9261 columns, 23604 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 3348 rows, 9261 columns and 23604 nonzeros
Model fingerprint: 0x51f4b159
Model has 4224 quadratic objective terms
Variable types: 1 continuous, 9260 integer (2924 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+01, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve remov

In [63]:
model3.pprint()

8 Set Declarations
    A : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {'Bochum', 'Boenen', 'Dortmund', 'Gelsenkirchen', 'Hagen', 'Iserlohn', 'Neuss', 'Schwerte'}
    A_f : Size=3, Index=F, Ordered=Insertion
        Key         : Dimen : Domain : Size : Members
           IJmuiden :     1 :    Any :    9 : {'Bochum', 'Boenen', 'Dortmund', 'Gelsenkirchen', 'Hagen', 'Iserlohn', 'Neuss', 'Schwerte', 'IJmuiden'}
              Segal :     1 :    Any :    9 : {'Bochum', 'Boenen', 'Dortmund', 'Gelsenkirchen', 'Hagen', 'Iserlohn', 'Neuss', 'Schwerte', 'Segal'}
        South Wales :     1 :    Any :    9 : {'Bochum', 'Boenen', 'Dortmund', 'Gelsenkirchen', 'Hagen', 'Iserlohn', 'Neuss', 'Schwerte', 'South Wales'}
    F : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'IJmuiden', 'Segal', 'South Wales'}
    F_A : Size=1, Index=None, Ordered=T

In [57]:
print("Valid v indices:")
for idx in valid_v_indices:
    print(idx)

Valid v indices:
('IJmuiden', 'Bochum', 'Bochum', 1)
('IJmuiden', 'Bochum', 'Boenen', 1)
('IJmuiden', 'Bochum', 'Dortmund', 1)
('IJmuiden', 'Bochum', 'Gelsenkirchen', 1)
('IJmuiden', 'Bochum', 'Hagen', 1)
('IJmuiden', 'Bochum', 'Iserlohn', 1)
('IJmuiden', 'Bochum', 'Neuss', 1)
('IJmuiden', 'Bochum', 'Schwerte', 1)
('IJmuiden', 'Bochum', 'IJmuiden', 1)
('IJmuiden', 'Boenen', 'Bochum', 1)
('IJmuiden', 'Boenen', 'Boenen', 1)
('IJmuiden', 'Boenen', 'Dortmund', 1)
('IJmuiden', 'Boenen', 'Gelsenkirchen', 1)
('IJmuiden', 'Boenen', 'Hagen', 1)
('IJmuiden', 'Boenen', 'Iserlohn', 1)
('IJmuiden', 'Boenen', 'Neuss', 1)
('IJmuiden', 'Boenen', 'Schwerte', 1)
('IJmuiden', 'Boenen', 'IJmuiden', 1)
('IJmuiden', 'Dortmund', 'Bochum', 1)
('IJmuiden', 'Dortmund', 'Boenen', 1)
('IJmuiden', 'Dortmund', 'Dortmund', 1)
('IJmuiden', 'Dortmund', 'Gelsenkirchen', 1)
('IJmuiden', 'Dortmund', 'Hagen', 1)
('IJmuiden', 'Dortmund', 'Iserlohn', 1)
('IJmuiden', 'Dortmund', 'Neuss', 1)
('IJmuiden', 'Dortmund', 'Schwerte